# Introduction #

In these exercises we'll explore some ways of improving training outcomes.

Load the *MovieLens* dataset.

# Network Capacity #

Let's say you were starting your model development with a simple linear network.

In [ ]:
model = keras.Sequential([
    layers.Dense(1)
])

You suspect this model is underfitting the training data, so you want to add capacity. Add another dense layer with 16 units. How does it fit?


In [ ]:
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(1)
])

Now let's add a second hidden layer. Was there an improvment?

In [ ]:
model = keras.Sequential([
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

# Early Stopping #

Now train the model with early stopping.

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=5, min_delta=0.001)

Did it solve the overfitting problem?

# Learning Rate Schedules #

Here's another useful callback.

You can often get lower loss by decreasing the learning rate during training. Let's define a learning rate scheduler and rerun the model from earlier.

First we need to rebuild it to start with an untrained model.

In [ ]:
model = keras.Sequential([
    layers.Dense(8, activation='relu', input_shape=[11]),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)
])
model.compile(
    optimizer='adam',
    loss='mae'
)

Now we can add the schedule using a "callback".

In [ ]:
lr_schedule = keras.callbacks.ReduceLROnPlateau()

model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=30,
    callbacks=[lr_schedule],
)

Did we see an improvement?

# Conclusion #
